In [1]:
from dataset.flame import FlameSatelite
from dataset.flame import FlameSatelite
from torch.utils.data import random_split
from torchvision.transforms import transforms
from pathlib import Path
import os

In [2]:
compose = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

In [3]:
dataset = FlameSatelite(download=True, root= "../../data/flame_satellite")

train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

Downloading...
From (original): https://drive.google.com/uc?id=1QXWOIxrv4Q2CccWsafpDh7miASZqWDbY
From (redirected): https://drive.google.com/uc?id=1QXWOIxrv4Q2CccWsafpDh7miASZqWDbY&confirm=t&uuid=45719115-9cd1-41c9-86e8-f3de964e6e29
To: /Users/fauzanghaza/Downloads/gelatik-datamining-2025/data/flame_satellite/images.zip
100%|██████████| 43.0M/43.0M [00:08<00:00, 5.34MB/s]


Downloaded file to ../../data/flame_satellite/images.zip
Extracting images...
Found a directory: __MACOSX. Skipping...
Removing unwanted __MACOSX folder...
Found a nested 'images' folder. Flattening structure...


Downloading...
From: https://drive.google.com/uc?id=1KXJzjXNguRnT59-XlQhLomT-xfsjTYis
To: /Users/fauzanghaza/Downloads/gelatik-datamining-2025/data/flame_satellite/annotations.csv
100%|██████████| 2.16M/2.16M [00:00<00:00, 4.71MB/s]


Downloaded file to ../../data/flame_satellite/annotations.csv


In [4]:
root_dir = Path("../../data/flame_satellite")
train_img_dir = root_dir / "train/images"
train_lbl_dir = root_dir / "train/labels"
valid_img_dir = root_dir / "valid/images"
valid_lbl_dir = root_dir / "valid/labels"

for path in [train_img_dir, train_lbl_dir, valid_img_dir, valid_lbl_dir]:
    os.makedirs(path, exist_ok=True)

In [5]:
def convert_bbox_yolo(image_size, bbox):
    img_w, img_h = image_size
    x_min, y_min, x_max, y_max = bbox

    x_center = (x_min + x_max) / 2 / img_w
    y_center = (y_min + y_max) / 2 / img_h
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h

    return x_center, y_center, width, height

In [6]:
# 🔄 Function to save dataset in YOLO format
def save_yolo_format(dataset, img_dir, lbl_dir):
    for i, (image, bboxes) in enumerate(dataset):
        if image is None:
            continue  # Skip failed images

        img_name = f"img_{i}.jpg"
        lbl_name = f"img_{i}.txt"

        # 🖼 Save image
        img_path = img_dir / img_name
        image.save(img_path)

        # 📝 Save label in YOLO format
        img_size = image.size  # (width, height)
        lbl_path = lbl_dir / lbl_name
        with open(lbl_path, "w") as f:
            for bbox in bboxes:
                x_center, y_center, width, height = convert_bbox_yolo(img_size, bbox)
                f.write(f"0 {x_center} {y_center} {width} {height}\n")  # Class '0' for fire

In [7]:
save_yolo_format(train_dataset, train_img_dir, train_lbl_dir)

save_yolo_format(valid_dataset, valid_img_dir, valid_lbl_dir)